In [1]:
import os
from datetime import datetime, timezone
from google.cloud import storage
from dotenv import load_dotenv
load_dotenv("/Users/paigegiese/SYG/landproDATA_code/misc-work/.env")
from openai import OpenAI
import logging
import json
import time
import random
import pandas as pd
from typing import TypedDict, List, Dict, Any, Optional
from pydantic import BaseModel, Field
from typing import TypedDict, List, Dict, Any, Optional, Annotated, Literal
oai_client = OpenAI(api_key =os.getenv("LANDPRO_OAI_KEY"))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/paigegiese/SYG/landproDATA_admin/landprodata-server-65e4697bdfe1.json'

client = storage.Client()
bucket = client.get_bucket("landprodata-files")
blobs = bucket.list_blobs(prefix="Subdivisions/")
cutoff = datetime(2025, 1, 1, tzinfo=timezone.utc)
recent_blobs = [blob for blob in blobs if blob.time_created and blob.time_created > cutoff and blob.name.endswith('txt')]


def get_blob(bucket, blob_name):
    blob = bucket.blob(blob_name)
    return blob


In [20]:
for blob in recent_blobs:
    print(blob.name, blob.time_created)

Subdivisions/10887 - ID - Payette - EVANS-DONOHO ADDITION.01.pdf.txt 2025-01-21 16:24:41.932000+00:00
Subdivisions/10991 - ID - Payette - GREEN HILL TRACT.01.pdf.txt 2025-01-21 16:32:27.019000+00:00
Subdivisions/11 - ID - Ada - EASTON.01.pdf.txt 2025-04-08 01:02:22.198000+00:00
Subdivisions/14757 - ID - Boise - LAZY J RANCH EST.01.pdf.txt 2025-01-18 23:06:09.112000+00:00
Subdivisions/14865 - ID - Boise - STEPHENSON'S SUBDIVISION.01.pdf.txt 2025-01-18 23:05:48.114000+00:00
Subdivisions/15548 - ID - Clearwater - CLEARWATER PLATEAU SUBDIVISION.01.pdf.txt 2025-01-21 15:28:35.849000+00:00
Subdivisions/15556 - ID - Clearwater - CHASE TRACTS LOTS 6 AND 7.01.pdf.txt 2025-01-21 15:35:59.944000+00:00
Subdivisions/15603 - ID - Clearwater - HENGEN SUBDIVISION.01.pdf.txt 2025-01-21 15:37:59.578000+00:00
Subdivisions/16104 - ID - Bannock - LAVA HILLS ESTATES, DIVISION NO 1.01.pdf.txt 2025-01-21 18:57:16.612000+00:00
Subdivisions/29451 - ID - Twin Falls - MIKE GREEN MISC PLAT.01.pdf.txt 2025-01-21 16

In [3]:
class Person(BaseModel):
    first_name: Annotated[str, Field(description="A person's first name")]
    last_name: Annotated[str, Field(description="A person's last name")]
    title: Annotated[str, Field(description="A person's job title")]
    license_number: Annotated[str, Field(description="A person's license number, if applicable (likely for plats and surveys)")]

class Entities(BaseModel):
    name: Annotated[str, Field(description="The entity's full name")]
    type: Annotated[Literal['state', 'county', 'city','company','other'], Field(description="Identify entities in the document and categorize by type. Use 'other' if the existing types don't fit the context")]
    inferred_context: Annotated[str, Field(description="A 200-character description of why this entity is present. Why are they referenced? What appears to be the purpose of their presence?")]

class Document(BaseModel):
    people: Annotated[Optional[List[Person]], Field("Returns a list of people identified from the extracted text")] 
    entities: Annotated[Optional[List[Entities]], Field("Returns a list of non-human entities identified from the extracted text")] 
    tsr: Annotated[str, Field(description="Township, section, and range")]
    legal_description: Annotated[str, Field(description="Legal description from the extracted text")]
    confidence_in_ocr: Annotated[float, Field(description="Based on the text, how confident are you that the OCR process was of acceptable quality? 0-100 scale only.")]
    confidence_in_interpretation: Annotated[float, Field(description="Based on the text, how confident are you that you've interpreted the text appropriately? 0-100 scale only.")]

def ocr(text):
    response = oai_client.responses.parse(
        model="gpt-4.1",
        input=[
            {"role": "system", "content": "You are a helpful assistant at interpreting raw text extracted from subdivision plat maps and records of survey. \
             ##Precise instructions:\
             1. Carefully analyze the extracted text from the OCR process. The text is presented as one blob, but in its original form on the plat there are distinct sections. Keep this in mind\
             2. Review the required JSON structure for the response\
             3. Fulfill the request to the best of your ability\
             4. Reason about your true confidence in the OCR process output. Determine, on a 0-100 scale (100 = 100 percent certain of correctness) your confidence. Place the value in 'confidence_in_ocr' key \
             5. Reason about your true confidence in your interpretation. Determine, on a 0-100 scale (100 = 100 percent certain of correctness) your confidence. Place the value in 'confidence_in_interpretation' key\
             ## Formatting Instructions\
             Use as precise of language as possible: Only return necessary information, refrain from over-explanation or superfluous verbiage.\
             Only in 'inferred_context' are you allowed to practice freedom of explanation."},
            
            {"role": "user", "content": f"Return your interpretation of the following OCR text using the structured output model provided: {text}"}
        ],
        text_format = Document
    )
    return response


In [18]:
bb = get_blob(bucket, blob_name = "Subdivisions/5516 - ID - Ada - CYR'S SUBDIVISION IRELAND ADDITION BLOCKS 7 AND 9.01.pdf.txt").download_as_text()

In [19]:
print(bb)

unrecordeel
Contract of Sale
Edifet adjacent to and including Bik.q
FOR subordination Agreement
See Instrument No. 467422 Filed Nov 17 1997
Recorded Book 40 page_10_of_MissRecords
Pencil drawing: approximate.
84
CYR'S SUBDIVISION
OF
BLOCKS 7 AND 9.
IN
TRELAND ADDITION TO BOISE CITY,
In Sec. 34,T.4 N., R. 2 E.
SCALE 100 FT. TO I INCH.
Surveyed by C.C. Steverson
drawing
Approximate
Approvrinte drawin
33
! 35
-0.2 bop it go
& of See 33
J.R. Cyr
Ida Cyr
615 deg
1500
1
5.100.
12
96'
25
86'
26
50'
85
1
9
14
27
149
slea' 25.25)
2
COMMERCIAL
3
28
3
BOULEVARD 7
22
+
21
29
IC
30
5.9
21
STREET
$
a
20
7
52.90
We, R.Astansbury and J.R.Gyr hereby dedicate
to the use of the publid the streets and alleys showed hereon
in Blacks Yard 9.
A. Stansbury
R.9. Stansbury
State of Idaho
Courcy of olda
}s:6.
On this 27 day of January entre year 1905,
Before me John Hayres,aNotary Public ví andfos said
county personally appeared J.K.Grand (0.5. Starasburylenoun
to me to be the persons whose names are subscribed


In [20]:
response = ocr(bb)

In [27]:
response.output_parsed.confidence_in_ocr

60.0